In [1]:
import os
import plotly.graph_objects as go
from tqdm import tqdm
from gym import envs
import argparse
import numpy as np
from bigmdp.data.dataset import SimpleReplayBuffer, gather_data_in_buffer
from bigmdp.data.env_gym import SimpleNormalizeEnv, SimpleGymEnv, SimpleAtariEnv
from bigmdp.mdp.MDP_GPU_KDD import FullMDP
from bigmdp.utils.tmp_vi_helper import *
# from bigmdp.utils.tmp_vi_helper import get_iter_indexes
from bigmdp.utils.utils_directory import *
from bigmdp.utils.utils_log import *
from bigmdp.mdp.mdp_agent import SimpleAgent
from copy import deepcopy as cpy
from os import path
from arg_def import * 
import gym
import gym_cartpole_explore
import seaborn as sns
import wandb as wandb_logger 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.neighbors import KDTree
from IPython import display
import torch

In [2]:
# --log_wandb --project DFM_MDP --run_id default --ts_in_run_id 1 \
#         --algorithm PPO  --image_size 84 84 \
#         --data_version batch_rl --data_suffices 1 --sample_size 1 --test_size 0 \
#         --encoder_type none --bottleneck_size 16 --MAX_S_COUNT 350000\
#         --fill_with 0Q_src-KNN --mdp_build_k 1 --penalty_type linear --penalty_beta 0\
#         --build_mdp --save_mdp

In [3]:
parser, ArgumentDict = get_argument_parser()
options = "--env_name CartPole-v1 --action_repeat 1 --max_episode_length 250"
args = parser.parse_args(options.split(" "))


In [4]:
from image_wrappers import *

env = MaxLenNotDoneWrap(RepeatActionWrap(gym.make(args.env_name), args.action_repeat),
                                max_episode_length=args.max_episode_length)

In [8]:
train_buffer = SimpleReplayBuffer(200000)
train_buffer, info = gather_data_in_buffer(train_buffer, env, episodes = 99999999, policy = lambda s:np.random.randint(2), frame_count=200000, render = False )

Average Reward of collected trajectories:22.147
Data Collection Complete in 4.581763029098511 Seconds


In [9]:
class DummyNet():
    def __init__(self, sim, add_noise=False):
        self.simulator = sim
        self.use_cuda = False
        self.add_noise = add_noise

    def encode_single(self, o):
        return tuple(o)

    def encode_batch(self, o_batch):
        return [tuple(o) for o in o_batch]

    def predict_single_transition(self, o, a):
        return False

    def predict_batch_transition(self, o_batch, a_batch):
        return False


In [ ]:

empty_MDP = FullMDP(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    mdp_params={"weight_transitions": args.weight_transitions,
                            "mdp_build_k":args.mdp_build_k,
                            "plcy_lift_k": 1,
                            "knn_delta" : args.knn_delta,
                            "calc_action_vector":False,
                            "penalty_beta":args.penalty_beta,
                            "penalize_unknown_transitions": args.penalty_type != "none"
                            },
                    MAX_S_COUNT= args.MAX_S_COUNT,
                    MAX_NS_COUNT= args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent =  SimpleAgent(empty_MDP, net = DummyNet(None),  fill_with = args.fill_with,
                       sample_neighbors = False, pred_error_threshold = args.within_radius,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,
                       abstraction_flag=False, normalize_by_distance= args.normalize_by_distance)

mdp_id = "L[{}]-E[{}]-B[{}]-Fill[{}_K{}]-Penalty[{}_{}]".format(args.latent_type, args.encoder_type, args.bottleneck_size, 
                                                          args.fill_with, args.mdp_build_k, args.penalty_type, args.penalty_beta)

myAgent.build_mdp(train_buffer, batch_parse = True)




  4%|▎         | 29/782 [00:00<00:02, 288.65it/s]

Parsing Dataset Batch Parse: True



  0%|          | 578/200000 [00:00<00:34, 5779.95it/s]

Parsing Complete
Commiting Seen Transitions
Len of to commit transitions 200000
ABstraction Faldg False



 43%|████▎     | 85237/200000 [00:13<00:18, 6277.89it/s]

In [ ]:
myAgent.smoothing = True
myAgent.soft_q = False
myAgent.mdp_T.plcy_lift_k = 5
myAgent.mdp_T.smooth_with_seen = False

sum_reward_running = {policy_name:evaluate_on_env(env,  
                                                  policy, 
                                                  eps_count=50,progress_bar=True)[0]
                      for policy_name,policy in myAgent.policies.items()}


In [ ]:

for i in tqdm(range(int(1e8))):
    inner_loop_frame_count += 1
    outer_loop_frame_count += 1

    if args.render_every and eps_count % args.render_every == 0:
        env.render()

    if outer_loop_frame_count % params['bellman_backup_every'] == 0:
        st = time.time()
        mdp_T.do_optimal_backup(mode=args.device, n_backups=params["n_backups"])
        if args.rmax:
            mdp_T.do_explr_backup(mode=args.device, n_backups=params["n_backups"])
        bellman_backup_time.append(time.time() - st)

    policy_name, policy = "a", eps_opt_policy # list(explore_policies.items())[eps_count%len(explore_policies)]
    a = policy(s)

    ns, r, d, i = env.step(a)
    _d = False if d and i["max_episode_length_exceeded"] == True else d
    running_reward += r

    # add to buffer
    # Not necessary for Decimal Discretization bur sure
    exp = [s.tolist(), [a], ns.tolist(), [r], [_d]]
    train_buffer.add(exp, padded_info={"qval": 0})
    if np.random.randint(20) < 2:
        valid_buffer.add(exp, padded_info={"qval": 0})

    # Update MDP
    s_d, ns_d = img_2_disc_fxn([s]), img_2_disc_fxn([ns])
    hs_d, hns_d = hAsh(s_d.tolist()), hAsh(ns_d.tolist())
    mdp_T.consume_transition([hs_d, int(a), hns_d, float(r), int(_d)])
#     mdp_T.consume_transition((hAsh(hs.tolist()), int(a), hAsh(hns.tolist()), float(r), int(_d)))

    # prep for next loop
    s = ns

    # Housekeeping to omit while true loop
    if d:
        s = env.reset()
        running_reward = 0
        eps_count += 1

    if inner_loop_frame_count % params['evaluate_every'] == 0:
        mdp_T.do_optimal_backup(mode="GPU", n_backups=500)
        n_steps = outer_count * K_i + inner_loop_frame_count
        eval_reward = evaluate_on_env(test_env, opt_policy, eps_count=50, render=False)[0]
        
        col_header = ["Average Reward","MDP #States", "#MissingTrans/ #Trans","Missing %", "VI Error"] 
        meta_data = [10]*len(col_header) # max len of the data in the table to be printed (per column)
        print(' | '.join([s.ljust(max(meta_data[i], len(s)), '.') for i, s in enumerate(col_header)]))
        col_data = [eval_reward, 
                    len(mdp_T.tD),  
                    str(mdp_T.missing_state_action_count)+"/"+str(len(mdp_T.tD)*len(mdp_T.A)),
                    round(mdp_T.missing_state_action_count/mdp_T.total_state_action_count,4 ),
                    mdp_T.curr_vi_error       ]
        print(' | '.join([str(s).ljust(max(meta_data[i], len(col_header[i])),  " ") for i, s in enumerate(col_data)]))
        
        loggers_dict["tb_train_logger"].add_scalar("Optimal Avg Reward", n_steps)
        loggers_dict["tb_train_logger"].add_scalar("State Count", float(len(mdp_T.tD)), outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar('Mising Transition Count', mdp.missing_state_action_count, outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar("Opt VI Error",mdp_T.curr_vi_error, outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar("Safe VI Error",mdp_T.s_curr_vi_error, outer_loop_frame_count)

    ####################################################

    # Not necessary for Decimal Experiments
    # if outer_loop_frame_count % args.checkpoint_every == 0:
    #     training_net._save_to_cache()
    #     torch.save((train_buffer.buffer, mdp, net.state_dict()),
    #                log_dirs_dict["py_log_dir"] + "/checkpoint" + ".pth")

# Not necessary for Decimal Experiments
# training_net._save_to_cache()

  0%|          | 5007/100000000 [00:27<3085:17:30,  9.00it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
78.84          | 657         | 343/1314              | 0.261      | 0.39984130859375


  0%|          | 10007/100000000 [00:55<2869:20:35,  9.68it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
116.4          | 854         | 495/1708              | 0.2898     | 0.593780517578125


  0%|          | 15006/100000000 [01:22<3054:40:43,  9.09it/s]

...

  0%|          | 260006/100000000 [24:56<3022:08:58,  9.17it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
299.36         | 3578        | 1959/7156             | 0.2738     | 0.940338134765625


  0%|          | 265006/100000000 [25:28<3945:51:20,  7.02it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
294.58         | 3611        | 1974/7222             | 0.2733     | 1.4024658203125


  0%|          | 267577/100000000 [25:38<99:31:30, 278.36it/s] 